# Evaluating the Lighthouse Auditor Agent

This notebook demonstrates the capabilities of our Lighthouse auditor agent, which can:
1. Run Lighthouse audits on websites
2. Analyze performance, accessibility, and SEO issues
3. Generate and implement fixes via GitHub PRs

## Setup

In [ ]:
import os
import json
from app.agent import agent
from app.utils.lighthouse import LighthouseRunner, LighthouseConfig
from app.utils.github_integration import GitHubIntegration

# Optional: Set GitHub token for PR creation
# os.environ["GITHUB_TOKEN"] = "your-token-here"

## Test 1: Running a Lighthouse Audit

Let's test the agent's ability to run a Lighthouse audit on a website and analyze the results.

In [ ]:
messages = [
    {"role": "user", "content": "Run a Lighthouse audit on https://example.com and analyze the results"}
]

for chunk in agent.stream({"messages": messages}):
    if chunk.get("messages"):
        print(chunk["messages"][0].content)

## Test 2: Analyzing an Existing Report

Now let's test the agent's ability to analyze an existing Lighthouse report.

In [ ]:
# Example Lighthouse report (you would replace this with a real report)
sample_report = {
    "audits": {
        "performance": {
            "score": 0.7,
            "title": "Performance needs improvement",
            "description": "Several performance metrics are below target",
            "details": {
                "items": [
                    {
                        "suggestion": "Optimize image sizes",
                        "snippet": "<img src='large-image.jpg' />"
                    }
                ]
            }
        }
    }
}

messages = [
    {"role": "user", "content": f"Analyze this Lighthouse report and suggest improvements: {json.dumps(sample_report)}"}
]

for chunk in agent.stream({"messages": messages}):
    if chunk.get("messages"):
        print(chunk["messages"][0].content)

## Test 3: Creating GitHub PRs with Fixes

Finally, let's test the agent's ability to create pull requests with fixes (requires GitHub token).

In [ ]:
# Example fixes (you would get these from actual audit results)
example_fixes = [
    {
        "file_path": "index.html",
        "original_content": "<img src='large-image.jpg' />",
        "fixed_content": "<img src='large-image.jpg' loading='lazy' width='800' height='600' />",
        "description": "Added lazy loading and dimensions to improve performance",
        "issue_title": "Image Optimization"
    }
]

messages = [
    {"role": "user", "content": f"Create a pull request with these fixes for owner/repo: {json.dumps(example_fixes)}"}
]

for chunk in agent.stream({"messages": messages}):
    if chunk.get("messages"):
        print(chunk["messages"][0].content)

## End-to-End Workflow

Here's an example of a complete workflow: audit a site, analyze results, and create fixes.

In [ ]:
messages = [
    {"role": "user", "content": """Please help improve https://example.com:
    1. Run a Lighthouse audit
    2. Analyze the results
    3. Create a PR with fixes for owner/repo
    """}
]

for chunk in agent.stream({"messages": messages}):
    if chunk.get("messages"):
        print(chunk["messages"][0].content)

## Handling Large Reports

Demonstration of how the agent handles large Lighthouse reports by chunking them into manageable pieces.

In [ ]:
from app.utils.lighthouse import ReportChunker

# Example of a large report (you would have a real large report here)
large_report = {
    "metadata": {"timestamp": "2025-04-06T10:00:00.000Z"},
    "audits": {
        # ... many audit results ...
    }
}

# Chunk the report
chunks = ReportChunker.chunk_report(large_report)
print(f"Split report into {len(chunks)} chunks")

# Process each chunk
for i, chunk in enumerate(chunks):
    print(f"\nProcessing chunk {i+1}:")
    messages = [
        {"role": "user", "content": f"Analyze this part of the Lighthouse report: {json.dumps(chunk)}"}
    ]
    
    for response in agent.stream({"messages": messages}):
        if response.get("messages"):
            print(response["messages"][0].content)